In [ ]:
import zmq
import pandas as pd
import numpy as np
import sqlalchemy
from datetime import datetime
import time

import asyncio

from data import connect_db

begin = "2020"
end = "2023"

data_1 = database("database_1h")
data_1 = data_1.loc[begin:end]

data_2 = database("database_1h")
data_2 = data_2.loc[begin:end]

data_3 = database("database_1h")
data_3 = data_3.loc[begin:end]

In [ ]:

class Server_data():
    
    def __init__(self, data_1, data_2, data_3, sleep =2):
        self.data_1 = data_1
        self.data_2 = data_2
        self.data_3 = data_3
        
        self.bar = 0
        self.genere_data(sleep)
    
    
    def to_message(self, data, i, stop = 'false'):
        date = data.index[i]
        symbol = data['symbol'][0]
        date = date.timestamp()
        msg = '{} {} {} {} {} {} {} {}'.format(symbol, date,
                                    data.iloc[i]['open'], data.iloc[i]['high'],
                                    data.iloc[i]['low'], data.iloc[i]['close'],
                                    data.iloc[i]['volume'], stop
                                    )
        return msg
    
    
    
    def genere_data(self, sleep):
        context = zmq.Context()
        socket = context.socket(zmq.PUB)
        socket.bind('tcp://127.0.0.1:8080')
        
        while True:
            try:
                msg_1 = self.to_message(self.data_1, self.bar)
                msg_2 = self.to_message(self.data_2, self.bar)
                msg_3 = self.to_message(self.data_3, self.bar)
            except:
                msg = " ... end ... "
                print(msg)
                break
            
            print(msg_1)
            print(msg_2)
            print(msg_3)
            socket.send_string(msg_1)
            socket.send_string(msg_2)
            socket.send_string(msg_3)
            
            self.bar += 1
            time.sleep(sleep)
            
            
server = Server_data(data_1, data_2, data_3, sleep = 1)
